# PyMongoArrow: Bridging the Gap Between MongoDB and Your Data Analysis App

The code in this notebook requires you to have created a MongoDB Atlas cluster, with the sample datasets loaded. You will then need to set a `MDB_URI` environment variable with the URI of your cluster before starting Jupyter Notebook to run this.

## Getting Set Up:

In [1]:
# Install the dependencies for the code in the later code blocks:

import sys
!{sys.executable} -m pip install pymongo[srv]==3.12 pymongoarrow==0.1.1 pandas==1.3.3 numpy==1.21.2

You should consider upgrading via the '/home/judy2k/Documents/Development/pymongoarrow_post/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
# Import required modules:
import os
import pyarrow
import pymongo
import bson
import pymongoarrow.monkey
from pymongoarrow.api import Schema
import pandas as pd

# MDB_URI should be set in your terminal before starting Jupyter Notebook.
MDB_URI = os.environ['MDB_URI']

# Add extra find_* methods to pymongo collection objects:
pymongoarrow.monkey.patch_all()

# Connect to the MongoDB cluster:
client = pymongo.MongoClient(MDB_URI)
database = client.get_database("sample_weatherdata")
collection = database.get_collection("data")

In [3]:
# Look at the data in "sample_weatherdata":
list(collection.find(
    {'_id': bson.ObjectId("5553a998e4b02cf7151190bf")}))

[{'_id': ObjectId('5553a998e4b02cf7151190bf'),
  'st': 'x+49700-055900',
  'ts': datetime.datetime(1984, 3, 5, 15, 0),
  'position': {'type': 'Point', 'coordinates': [-55.9, 49.7]},
  'elevation': 9999,
  'callLetters': 'SCGB',
  'qualityControlProcess': 'V020',
  'dataSource': '4',
  'type': 'FM-13',
  'airTemperature': {'value': -5.1, 'quality': '1'},
  'dewPoint': {'value': 999.9, 'quality': '9'},
  'pressure': {'value': 1020.8, 'quality': '1'},
  'wind': {'direction': {'angle': 100, 'quality': '1'},
   'type': 'N',
   'speed': {'rate': 3.1, 'quality': '1'}},
  'visibility': {'distance': {'value': 20000, 'quality': '1'},
   'variability': {'value': 'N', 'quality': '9'}},
  'skyCondition': {'ceilingHeight': {'value': 22000,
    'quality': '1',
    'determination': 'C'},
   'cavok': 'N'},
  'sections': ['AG1', 'AY1', 'GF1', 'MD1', 'MW1'],
  'precipitationEstimatedObservation': {'discrepancy': '2',
   'estimatedWaterDepth': 0},
  'pastWeatherObservationManual': [{'atmosphericCondition'

In [4]:
# Use find_pandas_all to extract the ts field into a Pandas DataFrame:
collection.find_pandas_all(
    {},
    schema=Schema({
        'ts': pyarrow.timestamp('ms'),
    })
)

,ts
0,1984-03-05 15:00:00
1,1984-03-05 18:00:00
2,1984-03-05 18:00:00
3,1984-03-05 18:00:00
4,1984-03-05 18:00:00
...,...
9995,1984-03-13 06:00:00
9996,1984-03-13 06:00:00
9997,1984-03-13 06:00:00
9998,1984-03-12 09:00:00


In [5]:
# Use aggregate to flatten nested data:
list(collection.aggregate([
    {'$match': {'_id': bson.ObjectId("5553a998e4b02cf7151190bf")}},
    {'$project': {
        'windDirection': '$wind.direction.angle',
        'windSpeed': '$wind.speed.rate',
    }}
]))

[{'_id': ObjectId('5553a998e4b02cf7151190bf'),
  'windDirection': 100,
  'windSpeed': 3.1}]

In [6]:
# Use the same pipeline above, but this time extract the data into a Pandas DataFrame:
collection.aggregate_pandas_all([
        {'$project': {
            'windDirection': '$wind.direction.angle',
            'windSpeed': '$wind.speed.rate',
        }}
    ],
    schema=Schema({'windDirection': int, 'windSpeed': float})
)

,windDirection,windSpeed
0,100,3.1
1,50,9.0
2,30,7.7
3,270,19.0
4,50,8.2
...,...,...
9995,10,7.0
9996,60,5.7
9997,330,3.0
9998,140,7.7


In [7]:
# Let's do some benchmarking:
import timeit

print("PyMongoArrow Benchmark: ", timeit.timeit(lambda: collection.aggregate_pandas_all([
        {'$project': {
            'windDirection': '$wind.direction.angle',
            'windSpeed': '$wind.speed.rate',
        }}
    ],
    schema=Schema({'windDirection': int, 'windSpeed': float})
), number=20))

print("PyMongo Benchmark: ", timeit.timeit(lambda: pd.DataFrame.from_records(collection.aggregate([
        {'$project': {
            'windDirection': '$wind.direction.angle',
            'windSpeed': '$wind.speed.rate',
        }}
    ]
)), number=20))

PyMongoArrow Benchmark:  4.608498300003703
PyMongo Benchmark:  5.516810900000564


In [8]:
# Use describe to summarize the wind data:
collection.aggregate_pandas_all([
        {'$project': {
            'windDirection': '$wind.direction.angle',
            'windSpeed': '$wind.speed.rate',
        }}
    ],
    schema=Schema({'windDirection': int, 'windSpeed': float})
).describe()

,windDirection,windSpeed
count,10000.000000,10000.000000
mean,236.816100,40.334220
std,237.428734,176.781233
min,10.000000,0.000000
25%,80.000000,4.600000
50%,180.000000,7.200000
75%,300.000000,10.800000
max,999.000000,999.900000
